## Terokai pengelompokan K-Means menggunakan R dan prinsip data Tidy.

### [**Kuiz sebelum kuliah**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Dalam pelajaran ini, anda akan belajar cara mencipta kelompok menggunakan pakej Tidymodels dan pakej lain dalam ekosistem R (kita akan panggil mereka kawan 🧑‍🤝‍🧑), serta set data muzik Nigeria yang anda import sebelum ini. Kita akan membincangkan asas K-Means untuk pengelompokan. Ingatlah bahawa, seperti yang anda pelajari dalam pelajaran sebelumnya, terdapat banyak cara untuk bekerja dengan kelompok dan kaedah yang anda gunakan bergantung pada data anda. Kita akan mencuba K-Means kerana ia adalah teknik pengelompokan yang paling biasa. Mari kita mulakan!

Istilah yang akan anda pelajari:

-   Skor Silhouette

-   Kaedah Elbow

-   Inertia

-   Varians

### **Pengenalan**

[K-Means Clustering](https://wikipedia.org/wiki/K-means_clustering) adalah kaedah yang berasal dari bidang pemprosesan isyarat. Ia digunakan untuk membahagikan dan memisahkan kumpulan data kepada `k kelompok` berdasarkan persamaan dalam ciri-ciri mereka.

Kelompok ini boleh divisualisasikan sebagai [diagram Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), yang merangkumi satu titik (atau 'benih') dan kawasan yang berkaitan dengannya.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografik oleh Jen Looper</figcaption>


Pengelompokan K-Means mempunyai langkah-langkah berikut:

1.  Saintis data bermula dengan menentukan bilangan kelompok yang dikehendaki untuk dicipta.

2.  Seterusnya, algoritma secara rawak memilih K pemerhatian daripada set data untuk dijadikan pusat awal bagi kelompok (iaitu, centroid).

3.  Seterusnya, setiap pemerhatian yang tinggal diberikan kepada centroid yang paling dekat dengannya.

4.  Seterusnya, purata baru bagi setiap kelompok dikira dan centroid dipindahkan ke purata tersebut.

5.  Setelah pusat-pusat dikira semula, setiap pemerhatian diperiksa semula untuk melihat sama ada ia mungkin lebih dekat dengan kelompok lain. Semua objek diberikan semula menggunakan purata kelompok yang dikemas kini. Langkah pemberian kelompok dan pengemaskinian centroid diulang secara iteratif sehingga pemberian kelompok berhenti berubah (iaitu, apabila penumpuan dicapai). Biasanya, algoritma akan berhenti apabila setiap iterasi baru menghasilkan pergerakan centroid yang tidak ketara dan kelompok menjadi statik.

<div>

> Perlu diingat bahawa disebabkan oleh pengacakan pemerhatian awal k yang digunakan sebagai centroid permulaan, kita boleh mendapatkan hasil yang sedikit berbeza setiap kali kita menggunakan prosedur ini. Atas sebab ini, kebanyakan algoritma menggunakan beberapa *permulaan rawak* dan memilih iterasi dengan WCSS terendah. Oleh itu, sangat disarankan untuk sentiasa menjalankan K-Means dengan beberapa nilai *nstart* untuk mengelakkan *optimum tempatan yang tidak diingini.*

</div>

Animasi pendek ini menggunakan [karya seni](https://github.com/allisonhorst/stats-illustrations) oleh Allison Horst menerangkan proses pengelompokan:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>



Satu soalan asas yang timbul dalam pengelompokan ialah: bagaimana anda tahu berapa banyak kelompok untuk memisahkan data anda? Satu kelemahan menggunakan K-Means termasuk fakta bahawa anda perlu menetapkan `k`, iaitu bilangan `centroid`. Nasib baik, `kaedah elbow` membantu untuk menganggarkan nilai permulaan yang baik untuk `k`. Anda akan mencubanya sebentar lagi.

### 

**Prasyarat**

Kita akan bermula dari tempat kita berhenti dalam [pelajaran sebelumnya](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), di mana kita menganalisis set data, membuat banyak visualisasi dan menapis set data kepada pemerhatian yang menarik. Pastikan anda menyemaknya!

Kita memerlukan beberapa pakej untuk menyelesaikan modul ini. Anda boleh memasangnya dengan: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Sebagai alternatif, skrip di bawah akan memeriksa sama ada anda mempunyai pakej yang diperlukan untuk menyelesaikan modul ini dan memasangnya untuk anda sekiranya ada yang hilang.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Mari kita mulakan dengan pantas!

## 1. Tarian dengan data: Tumpukan kepada 3 genre muzik paling popular

Ini adalah ringkasan apa yang telah kita lakukan dalam pelajaran sebelumnya. Jom kita analisis dan pecahkan data!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Itu berjalan dengan baik!

## 2. Lebih banyak penerokaan data.

Seberapa bersih data ini? Mari kita periksa pencilan menggunakan plot kotak. Kita akan memberi tumpuan kepada lajur berangka dengan pencilan yang lebih sedikit (walaupun anda boleh membersihkan pencilan tersebut). Plot kotak boleh menunjukkan julat data dan akan membantu memilih lajur mana yang hendak digunakan. Perlu diingat, plot kotak tidak menunjukkan varians, elemen penting untuk data yang boleh dikelompokkan dengan baik. Sila lihat [perbincangan ini](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) untuk bacaan lanjut.

[Plot kotak](https://en.wikipedia.org/wiki/Box_plot) digunakan untuk menggambarkan secara grafik taburan data `berangka`, jadi mari kita mulakan dengan *memilih* semua lajur berangka bersama-sama dengan genre muzik popular.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Lihat bagaimana pembantu pemilihan `where` memudahkan perkara ini 💁? Terokai fungsi-fungsi lain seperti ini [di sini](https://tidyselect.r-lib.org/).

Oleh kerana kita akan membuat kotak plot untuk setiap ciri numerik dan kita ingin mengelakkan penggunaan gelung, mari kita susun semula data kita ke dalam format yang *lebih panjang* yang akan membolehkan kita memanfaatkan `facets` - subplot yang masing-masing memaparkan satu subset data.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Lebih panjang lagi! Sekarang masa untuk beberapa `ggplots`! Jadi, `geom` apa yang akan kita gunakan?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Mudah-gg!

Sekarang kita dapat lihat data ini agak berisik: dengan memerhatikan setiap lajur sebagai kotak plot, anda boleh nampak nilai luar. Anda boleh melalui set data ini dan membuang nilai luar tersebut, tetapi itu akan menjadikan data agak terhad.

Buat masa ini, mari kita pilih lajur mana yang akan kita gunakan untuk latihan pengelompokan kita. Mari kita pilih lajur berangka dengan julat yang serupa. Kita boleh mengekodkan `artist_top_genre` sebagai nilai berangka tetapi kita akan abaikan buat masa ini.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Mengira pengelompokan k-means dalam R

Kita boleh mengira k-means dalam R menggunakan fungsi terbina dalam `kmeans`, lihat `help("kmeans()")`. Fungsi `kmeans()` menerima bingkai data dengan semua lajur bernombor sebagai argumen utamanya.

Langkah pertama apabila menggunakan pengelompokan k-means adalah untuk menentukan bilangan kelompok (k) yang akan dihasilkan dalam penyelesaian akhir. Kita tahu terdapat 3 genre lagu yang telah kita asingkan daripada set data, jadi mari kita cuba 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Objek kmeans mengandungi beberapa maklumat yang dijelaskan dengan baik dalam `help("kmeans()")`. Buat masa ini, mari kita fokus pada beberapa perkara. Kita dapat lihat bahawa data telah dikelompokkan kepada 3 kluster dengan saiz 65, 110, 111. Hasil keluaran juga mengandungi pusat kluster (purata) untuk 3 kumpulan merentasi 5 pemboleh ubah.

Vektor pengelompokan adalah tugasan kluster untuk setiap pemerhatian. Mari kita gunakan fungsi `augment` untuk menambah tugasan kluster ke dalam set data asal.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Hebat, kita baru sahaja membahagikan set data kita kepada 3 kumpulan. Jadi, sejauh mana baiknya pengelompokan kita 🤷? Mari kita lihat pada `Silhouette score`.

### **Silhouette score**

[Analisis Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) boleh digunakan untuk mengkaji jarak pemisahan antara kluster yang dihasilkan. Skor ini berbeza dari -1 hingga 1, dan jika skor menghampiri 1, kluster tersebut adalah padat dan terpisah dengan baik daripada kluster lain. Nilai yang hampir dengan 0 menunjukkan kluster yang bertindih dengan sampel yang sangat dekat dengan sempadan keputusan kluster jiran. [sumber](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Kaedah silhouette purata mengira purata silhouette bagi pemerhatian untuk nilai *k* yang berbeza. Skor silhouette purata yang tinggi menunjukkan pengelompokan yang baik.

Fungsi `silhouette` dalam pakej cluster digunakan untuk mengira lebar silhouette purata.

> Silhouette boleh dikira dengan mana-mana [metrik jarak](https://en.wikipedia.org/wiki/Distance "Distance"), seperti [jarak Euclidean](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") atau [jarak Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") yang telah kita bincangkan dalam [pelajaran sebelumnya](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Skor kita ialah **.549**, jadi berada di tengah-tengah. Ini menunjukkan bahawa data kita tidak begitu sesuai untuk jenis pengelompokan ini. Mari kita lihat sama ada kita boleh mengesahkan andaian ini secara visual. [Pakej factoextra](https://rpkgs.datanovia.com/factoextra/index.html) menyediakan fungsi (`fviz_cluster()`) untuk memvisualisasikan pengelompokan.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Tumpang tindih dalam kelompok menunjukkan bahawa data kita tidak begitu sesuai untuk jenis pengelompokan ini, tetapi mari kita teruskan.

## 4. Menentukan jumlah kelompok yang optimum

Satu persoalan asas yang sering timbul dalam pengelompokan K-Means ialah - tanpa label kelas yang diketahui, bagaimana anda tahu berapa banyak kelompok untuk memisahkan data anda?

Salah satu cara untuk mencuba mengetahuinya adalah dengan menggunakan sampel data untuk `mencipta siri model pengelompokan` dengan bilangan kelompok yang meningkat (contohnya dari 1-10), dan menilai metrik pengelompokan seperti **Silhouette score.**

Mari kita tentukan bilangan kelompok yang optimum dengan mengira algoritma pengelompokan untuk nilai *k* yang berbeza dan menilai **Within Cluster Sum of Squares** (WCSS). Jumlah WCSS mengukur kepadatan pengelompokan dan kita mahu ia sekecil mungkin, dengan nilai yang lebih rendah bermaksud titik data lebih rapat.

Mari kita terokai kesan pilihan `k` yang berbeza, dari 1 hingga 10, terhadap pengelompokan ini.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Sekarang kita mempunyai jumlah keseluruhan dalam-kluster sum-of-squares (tot.withinss) untuk setiap algoritma pengelompokan dengan pusat *k*, kita menggunakan [kaedah siku](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) untuk mencari bilangan kluster yang optimum. Kaedah ini melibatkan melukis WCSS sebagai fungsi bilangan kluster, dan memilih [siku lengkung](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Siku lengkung") sebagai bilangan kluster yang akan digunakan.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Plot menunjukkan pengurangan besar dalam WCSS (jadi *ketatannya* lebih tinggi) apabila bilangan kluster meningkat dari satu ke dua, dan pengurangan yang lebih ketara lagi dari dua ke tiga kluster. Selepas itu, pengurangan menjadi kurang jelas, menghasilkan satu `elbow` 💪 pada carta sekitar tiga kluster. Ini adalah petunjuk yang baik bahawa terdapat dua hingga tiga kluster data yang agak terpisah dengan baik.

Kita kini boleh teruskan dan mengekstrak model pengelompokan di mana `k = 3`:

> `pull()`: digunakan untuk mengekstrak satu lajur
>
> `pluck()`: digunakan untuk mengindeks struktur data seperti senarai


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Bagus! Mari kita visualisasikan kluster yang diperoleh. Berminat untuk sedikit interaktiviti menggunakan `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Mungkin kita menjangkakan bahawa setiap kelompok (diwakili oleh warna yang berbeza) akan mempunyai genre yang berbeza (diwakili oleh bentuk yang berbeza).

Mari kita lihat ketepatan model ini.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Ketepatan model ini tidaklah buruk, tetapi tidak begitu baik. Mungkin data ini tidak sesuai untuk K-Means Clustering. Data ini terlalu tidak seimbang, kurang berkorelasi, dan terdapat terlalu banyak variasi antara nilai-nilai kolum untuk membentuk kluster dengan baik. Malah, kluster yang terbentuk mungkin sangat dipengaruhi atau berat sebelah oleh tiga kategori genre yang telah kita tentukan sebelum ini.

Walau bagaimanapun, ini adalah proses pembelajaran yang sangat menarik!

Dalam dokumentasi Scikit-learn, anda boleh lihat bahawa model seperti ini, dengan kluster yang tidak begitu jelas, mempunyai masalah 'variasi':

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografik daripada Scikit-learn</figcaption>



## **Variasi**

Variasi didefinisikan sebagai "purata perbezaan kuasa dua daripada Min" [sumber](https://www.mathsisfun.com/data/standard-deviation.html). Dalam konteks masalah pengelompokan ini, ia merujuk kepada data di mana nombor-nombor dalam set data kita cenderung menyimpang terlalu jauh daripada min.

✅ Ini adalah masa yang baik untuk memikirkan semua cara yang boleh anda gunakan untuk membetulkan isu ini. Ubah suai data sedikit lagi? Gunakan kolum yang berbeza? Gunakan algoritma yang berbeza? Petunjuk: Cuba [skala data anda](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) untuk menormalkannya dan uji kolum lain.

> Cuba '[kalkulator variasi](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' ini untuk memahami konsep ini dengan lebih mendalam.

------------------------------------------------------------------------

## **🚀Cabaran**

Luangkan masa dengan notebook ini, ubah suai parameter. Bolehkah anda meningkatkan ketepatan model dengan membersihkan data lebih banyak (contohnya, membuang outlier)? Anda boleh menggunakan berat untuk memberikan lebih banyak berat kepada sampel data tertentu. Apa lagi yang boleh anda lakukan untuk mencipta kluster yang lebih baik?

Petunjuk: Cuba skala data anda. Terdapat kod yang dikomen dalam notebook yang menambah skala standard untuk menjadikan kolum data lebih serupa dari segi julat. Anda akan dapati bahawa walaupun skor siluet menurun, 'kink' dalam graf siku menjadi lebih lancar. Ini kerana membiarkan data tidak berskala membolehkan data dengan variasi yang kurang membawa lebih banyak berat. Baca lebih lanjut tentang masalah ini [di sini](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Kuiz selepas kuliah**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Ulasan & Kajian Kendiri**

-   Lihatlah Simulator K-Means [seperti ini](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Anda boleh menggunakan alat ini untuk memvisualisasikan titik data sampel dan menentukan centroidnya. Anda boleh mengedit keacakan data, bilangan kluster dan bilangan centroid. Adakah ini membantu anda mendapatkan idea tentang bagaimana data boleh dikelompokkan?

-   Juga, lihat [handout tentang K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) dari Stanford.

Ingin mencuba kemahiran pengelompokan yang baru anda pelajari pada set data yang sesuai untuk K-Means clustering? Sila lihat:

-   [Latih dan Nilai Model Pengelompokan](https://rpubs.com/eR_ic/clustering) menggunakan Tidymodels dan rakan-rakan

-   [Analisis Kluster K-Means](https://uc-r.github.io/kmeans_clustering), Panduan Pemrograman Analitik Perniagaan UC R

- [Pengelompokan K-Means dengan prinsip data yang teratur](https://www.tidymodels.org/learn/statistics/k-means/)

## **Tugasan**

[Cuba kaedah pengelompokan yang berbeza](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## TERIMA KASIH KEPADA:

[Jen Looper](https://www.twitter.com/jenlooper) kerana mencipta versi Python asal modul ini ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) kerana mencipta ilustrasi yang menakjubkan yang menjadikan R lebih mesra dan menarik. Cari lebih banyak ilustrasi di [galerinya](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Selamat Belajar,

[Eric](https://twitter.com/ericntay), Duta Pelajar Microsoft Learn Emas.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
